# 1. Découverte des données

## 1.1 Installation des Packages

In [ ]:
pip install requests 

In [ ]:
pip install lxml 

In [ ]:
pip install selenium 

* requests est un package servant à récupérer les données d'un site web
* lxml est un package servant à télécharger des fichiers lxml
* selenium est un package servant à l'interaction automatisée avec un serveur

## 1.2 Importation des Librairies

In [ ]:
import lxml
import selenium
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests
sns.set(rc={'figure.figsize':(20,10)})
sns.set_theme()


## 1.3 Création d'une première base "Races"

* Récupération des données à partir d'une API en ligne : "ergast.com"
* Objectif : création d'une base "Races" regroupant les années des saisons, le nom et le rang du circuit dans la saison, la latitude et la longitude du circuit (utile plus tard pour les représentations géographiques), le pays et la date du circuit, ainsi que l'url wikipédia associé
* La base contient 550 lignes correspondant aux saisons comprises entre 1990 et 2020

In [ ]:
races = {'season': [],
        'round': [],
        'circuit_id': [],
        'lat': [],
        'long': [],
        'country': [],
        'date': [],
        'url': []}

for year in list(range(1990,2021)):
    
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        try:
            races['season'].append(int(item['season']))
        except:
            races['season'].append(None)

        try:
            races['round'].append(int(item['round']))
        except:
            races['round'].append(None)

        try:
            races['circuit_id'].append(item['Circuit']['circuitId'])
        except:
            races['circuit_id'].append(None)

        try:
            races['lat'].append(float(item['Circuit']['Location']['lat']))
        except:
            races['lat'].append(None)

        try:
            races['long'].append(float(item['Circuit']['Location']['long']))
        except:
            races['long'].append(None)

        try:
            races['country'].append(item['Circuit']['Location']['country'])
        except:
            races['country'].append(None)

        try:
            races['date'].append(item['date'])
        except:
            races['date'].append(None)

        try:
            races['url'].append(item['url'])
        except:
            races['url'].append(None)
        
race= pd.DataFrame(races)

In [ ]:
race

In [ ]:
race.info()

## 1.4 Création d'une deuxième base "Results"

* Objectif : création d'une deuxième base incluant notamment la position sur la grille de départ et le podium final
* Observations de valeurs manquantes sur la variable "time" : on choisit donc de l'exclure (64% de valeurs manquantes)

In [ ]:
rounds = []
for year in np.array(race.season.unique()):
    rounds.append([year, list(race[race.season == year]['round'])])

print(rounds)
# query API
    
results = {'season': [],
          'round':[],
           'circuit_id':[],
          'driver': [],
           'date_of_birth': [],
           'nationality': [],
          'constructor': [],
          'grid': [],
          'time': [],
          'status': [],
          'points': [],
          'podium': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races'][0]['Results']:
            try:
                results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            except:
                results['season'].append(None)

            try:
                results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            except:
                results['round'].append(None)

            try:
                results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            except:
                results['circuit_id'].append(None)

            try:
                results['driver'].append(item['Driver']['driverId'])
            except:
                results['driver'].append(None)
            
            try:
                results['date_of_birth'].append(item['Driver']['dateOfBirth'])
            except:
                results['date_of_birth'].append(None)
                
            try:
                results['nationality'].append(item['Driver']['nationality'])
            except:
                results['nationality'].append(None)

            try:
                results['constructor'].append(item['Constructor']['constructorId'])
            except:
                results['constructor'].append(None)

            try:
                results['grid'].append(int(item['grid']))
            except:
                results['grid'].append(None)

            try:
                results['time'].append(int(item['Time']['millis']))
            except:
                results['time'].append(None)

            try:
                results['status'].append(item['status'])
            except:
                results['status'].append(None)

            try:
                results['points'].append(int(item['points']))
            except:
                results['points'].append(None)

            try:
                results['podium'].append(int(item['position']))
            except:
                results['podium'].append(None)

           
results = pd.DataFrame(results)

In [ ]:
results

In [ ]:
results.info()

In [ ]:
results_na=results.copy()
sns.heatmap(results_na.isna()) # On visualise l'emplacements des valeurs manquantes On a bcp de valeurs manquantes et un peu partout
results_na.isna()['time'].mean() # 64% de valeurs manquantes on ne va donc pas utilise

## 1.5 Nettoyage d'une colonne aux données manquantes

In [ ]:
results=results.drop(labels='time', axis=1) #suppression de la colonne aux données manquantes

In [ ]:
results

# 2. Corrélation entre la Grille de départ et le classement d'arrivée

## 2.1 Première étude de corrélation

In [ ]:
sns.relplot(x="grid", y="podium", data=results)

* Commentaires : A première vue, il n'y a pas de corrélations claires entre la position initiale sur la grille et la position d'arrivée

In [ ]:
results_2010=results.copy()
results_2010=results_2010[results_2010["season"]>=2010]
sns.relplot(x="grid", y="podium", data=results_2010[results_2010["season"]==2020],hue="circuit_id")

* Nous restreignons l'étude à l'année l'étude du lien pôle entre la postion initiale et la position d'arrivée par circuit en 2010
* On n'observe pas une corrélation encore très franche, mais une tendance se dessine

## 2.2 Création d'une fonction de corrélation entre la grille et le podium

In [ ]:
 def corr_grid_podium(annee):
    
    circuits=results_2010[results_2010["season"] ==annee]["circuit_id"].unique()
    df=results_2010[results_2010["season"] ==annee]
    les_correlations={}

    for circuit in circuits:
        df1=df[df["circuit_id"]==circuit][['grid',"podium"]]
        corr=df1.corr()["podium"][0]
        les_correlations[circuit]=corr
    
    df2=pd.DataFrame(list(les_correlations.items()),
                   columns=['circuit_id', 'correlation_grid_pod'])
    df2=df2.sort_values(by='correlation_grid_pod',ascending=False)

    print(df2)

    _,(ax1) = plt.subplots(ncols=1)
    sns.barplot(data=df2, x='circuit_id', y='correlation_grid_pod', ax=ax1, palette=sns.color_palette("icefire"))
    

In [ ]:
print(corr_grid_podium(2019))

* Observations : On observe une corrélation proche de 0.84 (proche de 1) pour le circuit de Monaco, ce qui est cohérent car les rues sont très étroites pour effectuer un dépassement. Au contraire, il est très facile de manoeuvrer sur le circuit de SPa, d'où une corrélation bien plus faible.

# 3. Corrélation entre le constructeur et les classemements sur la grille et à l'arrivée

## 3.1 Position Moyenne des constructeurs sur la grille et à l'arrivée

In [ ]:
results_constructeur=results.copy()
results_constructeur=results_constructeur[["constructor","podium","grid"]]

In [ ]:
results_constructeur.groupby("constructor").mean().sort_values(by="podium",ascending=True)

* Observations : Comme l'on pouvait le prévoir, les constructeurs Mercedes, Ferrari et RedBull ont la meilleure moyenne en terme de position initiale et de place à l'arrivée

## 3.2 Pourcentage de courses gagnées par constructeur depuis 1990

In [ ]:
results_constructeur_1=results_constructeur.copy()
results_constructeur_1=results_constructeur_1[results_constructeur_1["podium"]==1]
df=(results_constructeur_1.groupby("constructor").count())/len(results_constructeur_1)*100
df=df.reset_index()
_,(ax1) = plt.subplots(ncols=1)
sns.barplot(data=df, x='constructor', y='podium', ax=ax1, palette=sns.color_palette("icefire"))

* Observations : là encore, sans grande surprise, des constructeurs sortent du lot, comme Ferrari, McLaren, et Mercedes

# 4. Corrélation entre la nationalité du pilote et le pourcentage de victoires

In [ ]:
results_nationality=results.copy()
results_nationality=results_nationality[["nationality","grid","podium","circuit_id"]]

In [ ]:
nb_courses=len(results_nationality[results_nationality["podium"]==1])
df=((results_nationality[results_nationality["podium"]==1].groupby("nationality").count())/(nb_courses))*100
df=df.reset_index()
sns.barplot(x='nationality',y='podium',data=df,palette=sns.color_palette("icefire"))

* Observations : Sans surprise, les pilotes les plus titrés sur les grands prix sont de nationalité allemande (Vettel, Schumacher, Rosberg), Anglaise (Hamilton), Brésilienne (Senna) et FInlandaise (Räikkönen) 
* On va ajouter une colonne "nationality" et "country" à notre base

In [ ]:
df_country_race=race[["circuit_id","country"]].copy()
df_country_race.head()
results_merged=results.merge(df_country_race ,how='left', on="circuit_id")

In [ ]:
print(results_merged)
print(results_merged["nationality"].unique())
print(results_merged["country"].unique())

## 5. Corrélation entre l'âge et les victoires sur les grands prix

In [ ]:
results

In [ ]:
results_age=results.copy() #on copie la base results dans une nouvelle base qu'on va retravailler

results_age["date_of_birth"]=pd.to_datetime(results_age["date_of_birth"])

results_age["season"]=pd.to_datetime(results_age["season"], format="%Y") # conversion en format date

results_age["age"]=round(((results_age["season"]-results_age['date_of_birth']).dt.days)/365) #On déte

In [ ]:
results_age["date_of_birth"]

In [ ]:
results_age["season"]

In [ ]:
results_age["age"]

In [ ]:
df=results_age[["age","podium"]]
df_gagnant=df[df["podium"]==1]
sns.histplot(data=df_gagnant, x='age',kde=True) #je trace l'histogramme du nombre de victoire en fonction de l'âge.

* Observations : On observe une légère corrélation négative entre le nombre de victoires et l'âge du pilote mais elle n'est pas signficative

In [ ]:
corr = df.corr()
sns.heatmap(corr,annot=True,vmin=0, vmax=1)

In [ ]:
driver_standings = {'season': [],
                    'round':[],
                    'driver': [],
                    'driver_points': [],
                    'driver_wins': [],
                   'driver_standings_pos': []}

# query API

for n in list(range(len(rounds))):     
    for i in rounds[n][1]:    # iterate through rounds of each year
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            try:
                driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                driver_standings['season'].append(None)

            try:
                driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                driver_standings['round'].append(None)
                                         
            try:
                driver_standings['driver'].append(item['Driver']['driverId'])
            except:
                driver_standings['driver'].append(None)
            
            try:
                driver_standings['driver_points'].append(int(item['points']))
            except:
                driver_standings['driver_points'].append(None)
            
            try:
                driver_standings['driver_wins'].append(int(item['wins']))
            except:
                driver_standings['driver_wins'].append(None)
                
            try:
                driver_standings['driver_standings_pos'].append(int(item['position']))
            except:
                driver_standings['driver_standings_pos'].append(None)
            
driver_standings = pd.DataFrame(driver_standings)

# define lookup function to shift points and number of wins from previous rounds

def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df
  
driver_standings = lookup(driver_standings, 'driver', 'driver_points')
driver_standings = lookup(driver_standings, 'driver', 'driver_wins')
driver_standings = lookup(driver_standings, 'driver', 'driver_standings_pos')

driver_standings.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'], 
                      axis = 1, inplace = True)

In [ ]:
constructor_standings